In [1]:
import reboundx
import rebound
import mesa_reader as mr
from matplotlib import pyplot as plt
import numpy as np
import scipy.interpolate as interp
from astropy import constants as const
from astropy import units as u

In [2]:
msol = const.M_sun.to('g').value
rsol = const.R_sun.to('cm').value
au = const.au.to('cm').value
standard_cgrav = const.G.cgs.value
G = const.G.to('au3/(M_sun yr2)').value
M_jup = const.M_jup.to('M_sun').value
R_jup = const.R_jup.to('AU').value
secyer = u.yr.to(u.s)
aursun = u.R_sun.to(u.au)

In [3]:
def read_mesa(mesa_file):
    """ Read MESA history files with mesa_reader. """
    return mr.MesaData(mesa_file)
def interpolate_mesa_binary_history(key):
    """ Interpolate MESA binary history with 'age'.
    
    Parameter
    -----
    key: str
        variable names in MESA binary_history.data
    b: mesa_reader.MesaData
        loaded binary.data
    
    Return
    -----
    history_interp : scipy.interpolate.interpolate.interp1d
        linearly interpolated varibles 
    """
    
    age = b.data('age')
    history = b.data(key)
    history_interp = interp.interp1d(age,history)
    return history_interp

def interpolate_mesa_history(key,idx):
    """ Interpolate MESA star history with 'age'.
    
    Parameter
    -----
    key: str
        variable names in MESA history.data
    idx: int
        star index, 1, 2
    s: mesa_reader.MesaData
        loaded star.data
    
    Return
    -----
    history_interp : scipy.interpolate.interpolate.interp1d
        linearly interpolated varibles
    """
    if idx ==1:
        s =s1
    elif idx ==2:
        s =s2
    star_age = s.data('star_age')
    history = s.data(key)
    history_interp = interp.interp1d(star_age,history)
    return history_interp

def split_array(data):
    """ Split an array into groups with consecutive sequences. """
    return np.split(data, np.where(np.diff(data) != 1)[0]+1)

def find_mass_trasfer_phase(lg_mdot_benchmark):
    """ Find intervals of mass transfer phase.
    
    Parameter
    -----
    lg_mdot_benchmark: int
        minimal log10 mass change rate of the donor star that could be identified as mass transfer phase
    
    Return
    -----
    index_group : list
        list of indexes of mass transfer phase
    """
    lg_mstar_dot_1 = b.data('lg_mstar_dot_1')
    index_mdot = np.where(lg_mstar_dot_1 > lg_mdot_benchmark)
    index_group = split_array(index_mdot[0])
    return index_group

def time_mesa(t1,t2):
    """ get the MESA time sequence within input time range t1 and t2.
    
    Parameter
    -----
    t1: float
        start time of the simulation
    t2: float
        end time of the simulation
    x1: int
        index of the binary age that is the closest to t1
    x2: int
        index of the binary age that is the closest to t2
        
    Return
    -----
    times: ndarray
        MESA time sequence for the simulation including t1 and t2
    """
    x1 = np.searchsorted(age, t1)
    x2 = np.searchsorted(age, t2)-1
    times = age[np.arange(x1,x2)]
    if t1 != age[x1]:
        times = np.insert(times, 0, t1)
    if t2 != age[x2]:
        times = np.insert(times, len(times), t2)
    return times

def time_update_mesa(times, key, deltamax):
    """ re-calibrate the time sequence based on the threshold of the change of the input varible.
    
    Parameter
    -----
    times: ndarray
        MESA time sequence before calibrate
    key: str
        the varible that is used to refine the time steps
    deltamax: float
        the maximum change of the variable within one time step
        
    Return
    -----
    np.array(t): ndarray
        calibrated time sequence for the simulation, within each time step the change of the varible is smaller than 'deltamax'
    """
    t = []
    if key not in MESA_INPUT:
        raise Exception('key not in MESA_INPUT')
    arr = interpolate_mesa_binary_history(MESA_INPUT[key])(times)
    for i in range(len(times) - 1):
        if arr[i]-arr[i+1]> deltamax:
            n = int((arr[i]-arr[i+1])/deltamax + 2)
            t.extend(np.linspace(times[i],times[i+1],n).tolist()[:-1])
        else:
            t.append(times[i])
    t.append(times[-1])
    return np.array(t)

def calculate_kt_conv(M_env, DR_env, Renv_middle, Lum, W, M, n):
    """ calculate k/T based on Hurley et al. 2002 for convective envelope
    
    Parameter
    -----
    M_env: float
        mass of the dominant convective region for tides above the core
    DR_env: float
        thickness of the dominant convective region for tides above the core
    Renv_middle : float
        position of the dominant convective region for tides above the core
    Lum: float
        luminosity of the star
    W: float
        average spin angular velocity of the star
    M: float
        total mass of the star
    n: float
        mean orbital angular velocity
        
    Return
    -----
    kT_conv: float
        k/T for convective envelope
    """
    tau_conv = 0.431 * ((M_env * DR_env * Renv_middle/ (3 * Lum)) ** (1.0 / 3.0))
    P_spin = 2 * np.pi / W  
    P_orb = 2 * np.pi / n   
    P_tid = np.abs(1 / (1 / P_orb - 1 / P_spin))   
    f_conv = np.min([1, (P_tid / (2 * tau_conv)) ** 2])
    kT_conv = ((2. / 21) * (f_conv / tau_conv) * (M_env / M)) 
    return kT_conv
def calculate_kt_rad(conv_mx1_top_r, conv_mx1_bot_r, R, M1, M2, surface_h1, a): 
    """ calculate k/T based on Hurley et al. 2002, Qin et al. 2018 for radiative envelope
    
    Parameter
    -----
    conv_mx1_top_r: float
        coordinate of top convective mixing zone coordinate in Rsolar
    conv_mx1_bot_r : float
        coordinate of bottom convective mixing zone coordinate in Rsolar
    R: float
        radius of the star
    M1: float
        mass of the target star
    M2: float
        mass of the companion star
    surface_h1: float
        surface mass Hydrogen abundance
    a: float
        orbital semi-major axis
        
    Return
    -----
    kT_conv: float
        k/T for radiative envelope
    """
    R_conv = conv_mx1_top_r - conv_mx1_bot_r
    q = M2 / M1
    if (R_conv > R or R_conv <= 0.0 or conv_mx1_bot_r / R > 0.1):
        E2 = 1.592e-9 * M1 ** (2.84)
    else:
        if R <= 0:
            E2 = 0
        elif surface_h1 > 0.4:
            E2 = 10.0 ** (-0.42) * (R_conv / R) ** (7.5)  
        elif surface_h1 <= 0.4:  # "HeStar"
            E2 = 10.0 ** (-0.93) * (R_conv / R) ** (6.7)  
        else:  
            E2 = 1.592e-9 * M1 ** (2.84)      
    kT_rad = np.sqrt(standard_cgrav * (M1 * msol) * (R * rsol)**2 / (a * au)**5) * (1 + q) ** (5.0 / 6) * E2 * secyer
    return kT_rad


In [5]:
MESA_INPUT_1={
    # primary variables that go to REBOUND from binary history
    'm1':'star_1_mass',
    'r1':'star_1_radius',
    'mdot1':'lg_mstar_dot_1',
}
MESA_INPUT_2={
    # secondary and binary variables that go to REBOUND from binary history
    'm2':'star_2_mass',
    'r2':'star_2_radius',
    'a':'binary_separation',
}
MESA_INPUT  = {**MESA_INPUT_1, **MESA_INPUT_2}

MESA_INPUT_PRIMARY={
    # primary variables that go to REBOUND from star1 history
    'M_pri':'star_mass',
    'log_R_pri':'log_R',
    'W_pri':'surf_avg_omega',
    'log_L_pri':'log_total_angular_momentum',
    'I_pri':'total_moment_of_inertia',
    'M_env_pri':'mass_conv_reg_fortides',
    'DR_env_pri':'thickness_conv_reg_fortides',
    'Renv_middle_pri':'radius_conv_reg_fortides',
    'log_Lum_pri':'log_L',
    'conv_mx1_top_r_pri':'conv_mx1_top_r',
    'conv_mx1_bot_r_pri':'conv_mx1_bot_r',
    'surface_h1_pri':'surface_h1',
    
}
MESA_INPUT_SECONDARY={
    # secondary variables that go to REBOUND from star2 history
    'M_sec':'star_mass',
    'log_R_sec':'log_R',
    'W_sec':'surf_avg_omega',
    'log_L_sec':'log_total_angular_momentum',
    'I_sec':'total_moment_of_inertia',
    'M_env_sec':'mass_conv_reg_fortides',
    'DR_env_sec':'thickness_conv_reg_fortides',
    'Renv_middle_sec':'radius_conv_reg_fortides',
    'log_Lum_sec':'log_L',
    'conv_mx1_top_r_sec':'conv_mx1_top_r',
    'conv_mx1_bot_r_sec':'conv_mx1_bot_r',
    'surface_h1_sec':'surface_h1',
}


In [37]:
#load file path
binary_file = './binary_history.data'
star_file1 = './star1.data'
star_file2 = './star2.data'
kaps_pri_file = './kaps_pri.csv'
kaps_sec_file = './kaps_sec.csv'
kaps_pri_time_file = './kaps_pri_time.csv'
kaps_sec_time_file = './kaps_sec_time.csv'

b = read_mesa(binary_file)
s1 = read_mesa(star_file1)
s2 = read_mesa(star_file2)
age = b.data('age')
# get the linear interpolations for all variables
for key in MESA_INPUT:
    locals()[key] = interpolate_mesa_binary_history(MESA_INPUT[key])
for key in MESA_INPUT_PRIMARY:
    locals()[key] = interpolate_mesa_history(MESA_INPUT_PRIMARY[key],1)
for key in MESA_INPUT_SECONDARY:
    locals()[key] = interpolate_mesa_history(MESA_INPUT_SECONDARY[key],2)
# get the linear interpolations  for the apsidal motion constant
kaps_pri_data = np.loadtxt(kaps_pri_file)
kaps_pri_time = np.loadtxt(kaps_pri_time_file)
kaps_pri = interp.interp1d(kaps_pri_time, kaps_pri_data)
kaps_sec_data = np.loadtxt(kaps_sec_file)
kaps_sec_time = np.loadtxt(kaps_sec_time_file)
kaps_sec = interp.interp1d(kaps_sec_time, kaps_sec_data)    
    
# find mass transfer intervals with mass transfer rate larger than 10^{-6} Msun/yr
MT_interval = find_mass_trasfer_phase(-6)

# define the time interval for the simulation and refine the interval with deltamax=1e-5 for the mass change of the star1
t1 = age[MT_interval[0][0]]-5e6  #[0,max(age))
t2 = age[MT_interval[1][-1]]+1e8 #(t1,max(age)]
times = time_update_mesa(time_mesa(t1,t2),'m1',1e-5)

In [ ]:
# initial conditions
t_initial =  times[0]
M_pri_initial = M_pri(t_initial)
M_sec_initial = M_sec(t_initial) 
R_pri_initial = 10**log_R_pri(t_initial) * aursun
R_sec_initial = 10**log_R_sec(t_initial) * aursun
a_initial= a(t_initial) * aursun
I_pri_initial = I_pri(t_initial) / msol / au**2 
I_sec_initial = I_sec(t_initial) / msol / au**2
W_pri_initial = 10**log_L_pri(t_initial)/I_pri(t_initial) * secyer
W_sec_initial = 10**log_L_sec(t_initial)/I_sec(t_initial) * secyer
kaps_pri_initial = kaps_pri(t_initial)
kaps_sec_initial = kaps_sec(t_initial)

ap_initial = 0.5
ep_initial = 0.0
inc_initial = 0.0
integrator = 'whfast'

def makesim():
    sim = rebound.Simulation()
    sim.units = ('yr', 'AU', 'Msun') 
    sim.add(m=M_pri_initial,r=R_pri_initial)      
    sim.add(m=M_sec_initial,r=R_sec_initial,a=a_initial)
    sim.add(a=ap_initial,e=ep_initial,inc =inc_initial, m=M_jup, r=R_jup)
    sim.integrator = integrator 
    sim.move_to_com()
    return sim

sim = makesim()
ps = sim.particles
rebx = reboundx.Extras(sim)
sf = rebx.load_force("tides_spin")
rebx.add_force(sf)

ps[0].params['Omega'] = rebound.spherical_to_xyz(magnitude=W_pri_initial, theta=0, phi=0)
ps[0].params['I'] = I_pri_initial
ps[1].params['Omega'] = rebound.spherical_to_xyz(magnitude=W_sec_initial, theta=0, phi=0)
ps[1].params['I'] = I_sec_initial
ps[2].params['Omega'] = rebound.spherical_to_xyz(magnitude=2*np.pi/(ps[2].P), theta=0, phi=0)
ps[2].params['I'] = 0.25 * ps[2].m * ps[2].r**2
ps[0].params['k2'] = 2*kaps_pri_initial
ps[1].params['k2'] = 2*kaps_sec_initial
ps[2].params['k2'] = 0.565

# calculate k/T 
kT_conv_pri_initial = calculate_kt_conv(M_env_pri(t_initial),DR_env_pri(t_initial),
                                       Renv_middle_pri(t_initial),10**log_Lum_pri(t_initial),
                                       W_pri_initial,M_pri_initial,ps[2].n)
kT_conv_sec_initial = calculate_kt_conv(M_env_sec(t_initial),DR_env_sec(t_initial),
                                       Renv_middle_sec(t_initial),10**log_Lum_sec(t_initial),
                                       W_sec_initial,M_sec_initial,ps[2].n)
kT_rad_pri_initial = calculate_kt_rad(conv_mx1_top_r_pri(t_initial),conv_mx1_bot_r_pri(t_initial),
                                       10**log_R_pri(t_initial),M_pri(t_initial),M_sec(t_initial),
                                       surface_h1_pri(t_initial),ap_initial)
kT_rad_sec_initial = calculate_kt_rad(conv_mx1_top_r_sec(t_initial),conv_mx1_bot_r_sec(t_initial),
                                      10**log_R_sec(t_initial),M_sec(t_initial),M_pri(t_initial),
                                      surface_h1_sec(t_initial),ap_initial)
kT_pri_initial = max(kT_conv_pri_initial, kT_rad_pri_initial)
kT_sec_initial = max(kT_conv_sec_initial, kT_rad_sec_initial)

# lag time tau is related with T
ps[0].params['tau'] = kT_pri_initial/kaps_pri_initial*(R_pri_initial)**3/G/M_pri_initial
ps[1].params['tau'] = kT_sec_initial/kaps_sec_initial*(R_sec_initial)**3/G/M_sec_initial
# approximation for the planet
Q = 1e4
ps[2].params['tau'] = 1/(2*Q*ps[2].n)
rebx.initialize_spin_ode(sf)

# get interpolated values following the time sequence 
time_interval = times-t_initial
M_pri_interp = M_pri(times)
M_sec_interp = M_sec(times)
a_interp = a(times) * aursun
R_pri_interp = 10**log_R_pri(times) * aursun
R_sec_interp = 10**log_R_sec(times) * aursun
I_pri_interp = I_pri(times) / msol / au**2
I_sec_interp = I_sec(times) / msol / au**2
W_pri_interp = 10**log_L_pri(times)/I_pri(times) * secyer
W_sec_interp = 10**log_L_sec(times)/I_sec(times) * secyer

# variables to calculate k/T
M_env_pri_interp = M_env_pri(times)
M_env_sec_interp = M_env_sec(times)
DR_env_pri_interp = DR_env_pri(times)
DR_env_sec_interp = DR_env_sec(times)
Renv_middle_pri_interp = Renv_middle_pri(times)
Renv_middle_sec_interp = Renv_middle_sec(times)
log_Lum_pri_interp = log_Lum_pri(times)
log_Lum_sec_interp = log_Lum_sec(times)
log_R_pri_interp = log_R_pri(times)
log_R_sec_interp = log_R_sec(times)
conv_mx1_top_r_pri_interp = conv_mx1_top_r_pri(times)
conv_mx1_top_r_sec_interp = conv_mx1_top_r_sec(times)
conv_mx1_bot_r_pri_interp = conv_mx1_bot_r_pri(times)
conv_mx1_bot_r_sec_interp = conv_mx1_bot_r_sec(times)
surface_h1_pri_interp = surface_h1_pri(times)
surface_h1_sec_interp = surface_h1_sec(times)
kaps_pri_interp = kaps_pri(times)
kaps_sec_interp = kaps_sec(times)

# output list
Nout = len(time_interval)
a_p = np.zeros(Nout)
e_p = np.zeros(Nout)
inc_p = np.zeros(Nout)
x_p = np.zeros(Nout)
y_p = np.zeros(Nout)
z_p = np.zeros(Nout)
o_p = np.zeros((Nout, 3))

for i, t in enumerate(time_interval):

    sim.integrate(t)
    # update MESA quantities
    ps[0].m = M_pri_interp[i]
    ps[0].r = R_pri_interp[i]
    ps[0].params['I'] = I_pri_interp[i]
    ps[0].params['Omega'] = rebound.spherical_to_xyz(magnitude=W_pri_interp[i], theta=0, phi=0)
    ps[1].m = M_sec_interp[i]
    ps[1].r = R_sec_interp[i]
    ps[1].params['I'] = I_sec_interp[i]
    ps[1].params['Omega'] = rebound.spherical_to_xyz(magnitude=W_sec_interp[i], theta=0, phi=0)
    ps[1].a = a_interp[i]

    kT_conv_pri = calculate_kt_conv(M_env_pri_interp[i],DR_env_pri_interp[i],
                                    Renv_middle_pri_interp[i],10**log_Lum_pri_interp[i],
                                    W_pri_interp[i],M_pri_interp[i],ps[2].n)
    kT_conv_sec = calculate_kt_conv(M_env_sec_interp[i],DR_env_sec_interp[i],
                                    Renv_middle_sec_interp[i],10**log_Lum_sec_interp[i],
                                    W_sec_interp[i],M_sec_interp[i],ps[2].n)
    kT_rad_pri = calculate_kt_rad(conv_mx1_top_r_pri_interp[i],conv_mx1_bot_r_pri_interp[i],
                                  10**log_R_pri_interp[i],M_pri_interp[i],M_sec_interp[i],
                                  surface_h1_pri_interp[i],ps[2].a)  
    kT_rad_sec = calculate_kt_rad(conv_mx1_top_r_sec_interp[i],conv_mx1_bot_r_sec_interp[i],
                                  10**log_R_sec_interp[i],M_sec_interp[i],M_pri_interp[i],
                                  surface_h1_sec_interp[i],ps[2].a)
    kT_pri = max(kT_conv_pri, kT_rad_pri)
    kT_sec = max(kT_conv_sec, kT_rad_sec)
    
    ps[0].params['tau'] = kT_pri/kaps_pri_interp[i]*(R_pri_interp[i])**3/G/M_pri_interp[i]
    ps[1].params['tau'] = kT_sec/kaps_sec_interp[i]*(R_sec_interp[i])**3/G/M_sec_interp[i]
    ps[2].params['tau'] = 1/(2*Q*ps[2].n)
    ps[0].params['k2'] = 2*kaps_pri_interp[i]
    ps[1].params['k2'] = 2*kaps_sec_interp[i]

    a_p[i] = ps[2].a
    e_p[i] = ps[2].e
    inc_p[i] = ps[2].inc
    o_p[i] = ps[2].params['Omega']
    x_p[i] = ps[2].x
    y_p[i] = ps[2].y
    z_p[i] = ps[2].z
